# Panel Data

Sometimes, data comes in such a way that many observations share certain common features. For example, several measurements can be made in the same location, under the same condition, or for the same subject. To understand the data and extract meaningful insights, we often need to aggregate these observations. This is where the groupby() function comes into play.

## Loading

As always, let's start by importing pandas and loading and cleaning our dataset.

In [28]:
import pandas as pd

# Load the data
df = pd.read_csv("data/sp500_q1_2025.csv")

# Look at dates - back in the ISO format!
print("Look at our dates:\n", df.DlyCalDt.head())

# Convert the 'datadate' column to a datetime object
df.DlyCalDt = pd.to_datetime(df.DlyCalDt)

df


Look at our dates:
 0    2025-01-02
1    2025-01-03
2    2025-01-06
3    2025-01-07
4    2025-01-08
Name: DlyCalDt, dtype: object


,DlyCalDt,Ticker,SecurityNm,DlyOpen,DlyHigh,DlyLow,DlyClose,DlyVol,SICCD,PrimaryExch,PERMNO,PERMCO
0,2025-01-02,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,135.21,135.7300,132.870,133.43,953587,3826,N,87432,36364
1,2025-01-03,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,133.45,136.0500,132.755,135.69,1246919,3826,N,87432,36364
2,2025-01-06,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,135.60,138.3400,135.340,136.43,1047034,3826,N,87432,36364
3,2025-01-07,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,136.83,140.2800,135.980,137.41,1056693,3826,N,87432,36364
4,2025-01-08,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,137.68,137.6800,135.630,137.00,1684573,3826,N,87432,36364
...,...,...,...,...,...,...,...,...,...,...,...,...
29877,2025-03-25,ZTS,ZOETIS INC; COM A; CONS,163.31,164.3499,160.251,161.30,1583915,2834,N,13788,54327
29878,2025-03-26,ZTS,ZOETIS INC; COM A; CONS,161.16,163.4700,160.680,162.82,2078366,2834,N,13788,54327
29879,2025-03-27,ZTS,ZOETIS INC; COM A; CONS,163.25,165.4100,161.660,165.02,1590504,2834,N,13788,54327
29880,2025-03-28,ZTS,ZOETIS INC; COM A; CONS,164.00,165.3994,162.820,163.11,1550288,2834,N,13788,54327


We'll stop short of setting the index as our datetime value though. This is because an index must have unique values, and because this panel data contains lots of different company stocks for just one quarter of a year, we'll see the same date lots of times.

In [29]:
# How many unique dates exist in the data frame
print("Number of unique dates", df.DlyCalDt.nunique())

# Last date in the dataset
print("Last date", df.DlyCalDt.max())

# First date in the dataset
print("First date", df.DlyCalDt.min())


Number of unique dates 60
Last date 2025-03-31 00:00:00
First date 2025-01-02 00:00:00


## Cleaning

Let's not forget data cleaning! Do we have missing data? Where?

In [30]:
print("The number of missing values is", df.isnull().sum().sum())

df[df.isnull().any(axis=1)]

df.dropna(inplace=True)

print("The number of missing values is", df.isnull().sum().sum())

The number of missing values is 372
The number of missing values is 0


## Exploring

Let's explore this panel data a bit more, to answer some questions:

- How many tickers are considered
- How many securities are considered
- How many companies are considered
- Which exchanges are considered
- Which exchanges appear most


In [46]:
df.head()

print(df.Ticker.nunique())
print(df.PERMNO.nunique())
print(df.PERMCO.nunique())

print(df.PrimaryExch.unique())
print(df.PrimaryExch.value_counts())

494
498
491
['N' 'Q' 'B']
PrimaryExch
N    20369
Q     9360
B       60
Name: count, dtype: int64


## Grouping

What if we wanted to calculate daily returns in this data set. Is it as simple as using `pct_change()`? Let's try.

In [47]:
df["Return"] = df.DlyClose.pct_change()

df

,DlyCalDt,Ticker,SecurityNm,DlyOpen,DlyHigh,DlyLow,DlyClose,DlyVol,SICCD,PrimaryExch,PERMNO,PERMCO,Return
0,2025-01-02,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,135.21,135.7300,132.870,133.43,953587,3826,N,87432,36364,NaN
1,2025-01-03,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,133.45,136.0500,132.755,135.69,1246919,3826,N,87432,36364,0.016938
2,2025-01-06,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,135.60,138.3400,135.340,136.43,1047034,3826,N,87432,36364,0.005454
3,2025-01-07,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,136.83,140.2800,135.980,137.41,1056693,3826,N,87432,36364,0.007183
4,2025-01-08,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,137.68,137.6800,135.630,137.00,1684573,3826,N,87432,36364,-0.002984
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29877,2025-03-25,ZTS,ZOETIS INC; COM A; CONS,163.31,164.3499,160.251,161.30,1583915,2834,N,13788,54327,-0.011460
29878,2025-03-26,ZTS,ZOETIS INC; COM A; CONS,161.16,163.4700,160.680,162.82,2078366,2834,N,13788,54327,0.009423
29879,2025-03-27,ZTS,ZOETIS INC; COM A; CONS,163.25,165.4100,161.660,165.02,1590504,2834,N,13788,54327,0.013512
29880,2025-03-28,ZTS,ZOETIS INC; COM A; CONS,164.00,165.3994,162.820,163.11,1550288,2834,N,13788,54327,-0.011574


Can you see what's gone wrong here? Our first calculated daily return for American Airlines is using Agilent's last closing price. This hopefully gets across the importance of *grouping*, particularly useful with this kind of panel data.


We can solve this with the `groupby()` method of data frames.

In [49]:
df["Returns"] = df.groupby("PERMNO").DlyClose.pct_change()

df.head(65)

,DlyCalDt,Ticker,SecurityNm,DlyOpen,DlyHigh,DlyLow,DlyClose,DlyVol,SICCD,PrimaryExch,PERMNO,PERMCO,Return,Returns
0,2025-01-02,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,135.21,135.73,132.870,133.43,953587,3826,N,87432,36364,NaN,NaN
1,2025-01-03,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,133.45,136.05,132.755,135.69,1246919,3826,N,87432,36364,0.016938,0.016938
2,2025-01-06,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,135.60,138.34,135.340,136.43,1047034,3826,N,87432,36364,0.005454,0.005454
3,2025-01-07,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,136.83,140.28,135.980,137.41,1056693,3826,N,87432,36364,0.007183,0.007183
4,2025-01-08,A,AGILENT TECHNOLOGIES INC; COM NONE; CONS,137.68,137.68,135.630,137.00,1684573,3826,N,87432,36364,-0.002984,-0.002984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2025-01-02,AAL,AMERICAN AIRLINES GROUP INC; COM NONE; CONS,17.40,17.63,16.830,17.00,18765913,4512,Q,21020,20010,-0.854676,NaN
61,2025-01-03,AAL,AMERICAN AIRLINES GROUP INC; COM NONE; CONS,17.03,17.03,16.270,16.97,23623819,4512,Q,21020,20010,-0.001765,-0.001765
62,2025-01-06,AAL,AMERICAN AIRLINES GROUP INC; COM NONE; CONS,17.64,17.94,17.380,17.52,31580329,4512,Q,21020,20010,0.032410,0.032410
63,2025-01-07,AAL,AMERICAN AIRLINES GROUP INC; COM NONE; CONS,17.58,17.99,17.530,17.69,25289417,4512,Q,21020,20010,0.009703,0.009703


Perfect! Grouping is a very powerful way to manipulate panel data. Once you've grouped, you can call functions and they will be applied groupwise as we saw above. Here are some other common functions with groups:

In [53]:
df.groupby("Ticker").size()

apple = df.groupby("Ticker").get_group("AAPL")
apple

,DlyCalDt,Ticker,SecurityNm,DlyOpen,DlyHigh,DlyLow,DlyClose,DlyVol,SICCD,PrimaryExch,PERMNO,PERMCO,Return,Returns
120,2025-01-02,AAPL,APPLE INC; COM NONE; CONS,248.930,249.1000,241.8201,243.85,55236688,3571,Q,14593,7,22.113744,NaN
121,2025-01-03,AAPL,APPLE INC; COM NONE; CONS,243.360,244.1800,241.8900,243.36,39883569,3571,Q,14593,7,-0.002009,-0.002009
122,2025-01-06,AAPL,APPLE INC; COM NONE; CONS,244.310,247.3300,243.2000,245.00,44757842,3571,Q,14593,7,0.006739,0.006739
123,2025-01-07,AAPL,APPLE INC; COM NONE; CONS,242.980,245.5500,241.3500,242.21,40441122,3571,Q,14593,7,-0.011388,-0.011388
124,2025-01-08,AAPL,APPLE INC; COM NONE; CONS,241.920,243.7123,240.0500,242.70,38321555,3571,Q,14593,7,0.002023,0.002023
125,2025-01-10,AAPL,APPLE INC; COM NONE; CONS,240.010,240.1600,233.0000,236.85,61006510,3571,Q,14593,7,-0.024104,-0.024104
126,2025-01-13,AAPL,APPLE INC; COM NONE; CONS,233.530,234.6700,229.7200,234.40,49080156,3571,Q,14593,7,-0.010344,-0.010344
127,2025-01-14,AAPL,APPLE INC; COM NONE; CONS,234.750,236.1200,232.4720,233.28,38995805,3571,Q,14593,7,-0.004778,-0.004778
128,2025-01-15,AAPL,APPLE INC; COM NONE; CONS,234.635,238.9600,234.4300,237.87,39430187,3571,Q,14593,7,0.019676,0.019676
129,2025-01-16,AAPL,APPLE INC; COM NONE; CONS,237.350,238.0100,228.0300,228.26,70711015,3571,Q,14593,7,-0.040400,-0.040400


Let's see what else we can do with grouping. Recall that we had more tickers than companies. Let's see why that is by looking at how many unique tickers belong to each company (using `Ticker` and `PERMCO`). Then let's list those companies.

In [61]:
ticker_counts = df.groupby("PERMCO").Ticker.nunique()
multi_permco = ticker_counts[ticker_counts >1].index

df[df.PERMCO.isin(multi_permco)]



,DlyCalDt,Ticker,SecurityNm,DlyOpen,DlyHigh,DlyLow,DlyClose,DlyVol,SICCD,PrimaryExch,PERMNO,PERMCO,Return,Returns
11221,2025-01-02,FOX,FOX CORP; COM B; CONS,46.04,46.400,45.8200,46.24,972765,9999,Q,18421,56662,0.095994,NaN
11222,2025-01-03,FOX,FOX CORP; COM B; CONS,46.25,46.750,45.5900,46.72,842041,9999,Q,18421,56662,0.010381,0.010381
11223,2025-01-06,FOX,FOX CORP; COM B; CONS,47.11,47.590,46.9000,47.07,781300,9999,Q,18421,56662,0.007491,0.007491
11224,2025-01-07,FOX,FOX CORP; COM B; CONS,47.40,47.410,46.6350,46.81,861832,9999,Q,18421,56662,-0.005524,-0.005524
11225,2025-01-08,FOX,FOX CORP; COM B; CONS,46.85,46.850,46.0001,46.61,1206011,9999,Q,18421,56662,-0.004273,-0.004273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20576,2025-03-25,NWSA,NEWS CORP NEW; COM A; CONS,27.27,27.660,27.1900,27.46,2641756,9999,Q,13963,54433,0.005861,0.005861
20577,2025-03-26,NWSA,NEWS CORP NEW; COM A; CONS,27.51,27.745,27.3950,27.47,2852807,9999,Q,13963,54433,0.000364,0.000364
20578,2025-03-27,NWSA,NEWS CORP NEW; COM A; CONS,27.33,27.450,26.9200,26.94,3696957,9999,Q,13963,54433,-0.019294,-0.019294
20579,2025-03-28,NWSA,NEWS CORP NEW; COM A; CONS,26.97,27.520,26.4450,26.59,3221401,9999,Q,13963,54433,-0.012992,-0.012992


### Exercise: Tick Tick

**Part 1** Identify the number of unique tickers traded on each exchange.

In [31]:
## YOUR CODE GOES HERE

**Part 2** Then identify any securities that share a ticker.

In [32]:
## YOUR CODE GOES HERE

## Aggregation

Aggregation functions like `mean()`, `median()`, `sum()`, `min()`, `max()`, `first()`, `last()` and `std()` can be applied to grouped data to give insights across panel data. Say we wanted the average daily return of each traded security, or the max volume traded on any given day for each security?

The exercises above helped us identify that the `PERMNO` column corresponds to unique securities, so let's use that for grouping from now on. 

Useful, but only to a point. The `PERMNO` value is just a number to most of us. What if we want a ticker or name for the security? Let's look at grouping by multiple columns to help!

Once we've done these sorts of aggregation, we're often curious to see who sits at the top or the bottom of the distribution. We can use `nlargest()` and its antonym here. Note that `as_index=False` doesn't work here easily, since these functions refer to the index!

We can also group by multiple columns! This can be helpful when doing aggregation, for example, to find high performers in each month. Because our date is just a regular column, we need to specify `.dt` to use any datetime functions.

### Exercise: Good Days

Which two days of the week see the highest average close in this data set, and what is the average close for those days?  

In [33]:
## YOUR CODE GOES HERE

### Exercise: Trading Exchanges

Next identify the total trading volume of each exchange.

In [34]:
## YOUR CODE GOES HERE

### Exercise: The 1000 Club

For securities that reached a closing price above 1000, how many times in each month, did they acheive this?

In [35]:
## YOUR CODE GOES HERE

## Multiple Aggregation

We can use the `agg()` method, and pass it a dictionary to do multiple aggregations at once on grouped data. This can be helpful for further analyses, or for producing a more descriptive aggregated data frame.

### Quick Quarter Query

Using multiple aggregation, create an aggregated data frame with ticker and security name, the first open price in the period for each security and the last close price in the period for each security. Create a new column in this aggregated data frame that shows the price difference between final close and initial open for each security.

In [36]:
## YOUR CODE GOES HERE